# Analyzing (RE)PPTIS simulations using an MSM approach
This notebook contains an example workflow that can be used for estimating the crossing probability and pathlengths of a (RE)PPTIS simulation.

## 1. Import the necessary functions

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import matplotlib.pyplot as plt
from pprint import pprint    # to print the vars of the pathensemble object
import numpy as np
import os
import glob

# Reading
from tistools import read_inputfile, get_LMR_interfaces, read_pathensemble, get_weights
from tistools import set_tau_distrib, set_tau_first_hit_M_distrib, cross_dist_distr, pathlength_distr
from tistools import collect_tau, collect_tau1, collect_tau2, collect_taum
from tistools import ACCFLAGS, REJFLAGS

# REPPTIS analysis
from tistools import get_lmr_masks, get_generation_mask, get_flag_mask, select_with_masks
from tistools import unwrap_by_weight, running_avg_local_probs, get_local_probs, get_global_probs_from_dict, get_global_probs_from_local

# MSM functions
from tistools import construct_M
from tistools import global_pcross_msm
from tistools import mfpt_to_first_last_state, mfpt_to_absorbing_states, construct_tau_vector
from tistools import create_labels_states, print_vector, print_all_tau

## 2. Load the simulation data

### How to run this notebook
- Set `indir` to the simulation directory (see the commented presets below).
- Toggle `zero_minus_one` if you have a λ

−1 interface.
- Set `VERBOSE` to `True` if you want detailed per-pathensemble output; otherwise it stays concise.
- Figures are controlled by `makefigs` later on.
- The notebook assumes PyRETIS/REPPTIS output structure with numbered folders under `indir`.


In [ ]:
# Set the working directory
indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/simdata/infrepptis/"  


# Control verbosity and options
VERBOSE = True          # Set True for detailed per-ensemble printing
PRINT_PATHENSEMBLE = True  # Set True to pprint each pathensemble vars

# zero_minus_one: True if lambda_-1 interface is set
zero_minus_one = False

inputfile = indir + "/repptis.rst"    # When using PyRETIS, the input file for REPPTIS simulations is a .rst file

# Move to working directory
os.chdir(indir)
print(os.getcwd())

# Set the ensemble folders and print them
folders = sorted(glob.glob(indir + "/0[0-9][0-9]"))
print(f"Found {len(folders)} ensemble folders")
print(folders)


In [ ]:
# Reading all input
#===================
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)

pathensembles = []
for i, fol in enumerate(folders):
    pe = read_pathensemble(fol + "/pathensemble.txt")
    pe.set_name(fol)
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i == 0:
        pe.set_zero_minus_one(zero_minus_one)   # TODO this is never used
        pe.set_in_zero_minus(True)
    if i == 1:
        pe.set_in_zero_plus(True)

    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose=False)
    pe.set_weights(w)

    if PRINT_PATHENSEMBLE:
        print("#" * 80)
        print(fol)
        print("pathensemble info:")
        pprint(vars(pe))
    else:
        print(f"Loaded {fol} (ensemble {i})")

    # Read order parameters order.txt/order.npy into path ensemble object, or load from order.npy file.
    # Saving order parameter files allows to speed up this notebook.
    pe.set_orders(load=False, acc_only=True, save=True)        # first run: store .npy files
    # pe.set_orders(load=True, acc_only=True, save=False)          # subsequent runs: read .npy files
    # pe.set_orders(load=False, acc_only=True, save=False)       # if saving doesn't work

    pathensembles.append(pe)

print(f"\nLoaded {len(pathensembles)} pathensembles")


## 3. Regular (RE)PPTIS analysis using tistools

### Analyze the REPPTIS simulation.

**Tip on output volume**: Set `VERBOSE = True` (in the setup cell above) to see detailed per-ensemble diagnostics. With `VERBOSE = False`, only compact summaries are printed to keep the log tidy.

In [ ]:
# Analysis output is saved to the data dictionary.
data = {}
for i, pe in enumerate(pathensembles):
    if i == 0:
        data[i] = {}
        continue  # [0-] is not used for Pcross calculations

    if VERBOSE:
        print(f"doing pathensemble {i}")

    # Classify the paths according to their path type.
    pathtypes = ("LML", "LMR", "RML", "RMR")
    pathtype_cycles = {}
    for ptype in pathtypes:
        pathtype_cycles[ptype] = unwrap_by_weight(
            (pe.lmrs == ptype).astype(int), pe.weights
        )
    
    # Running average analysis: ["running"]
    data[i] = {}
    data[i]["running"] = {}
    data[i]["running"]["plocal"] = {}
    for (ptype, p_loc) in zip(
        pathtypes,
        running_avg_local_probs(pathtype_cycles, pe.weights, tr=False)
    ):
        data[i]["running"]["plocal"][ptype] = p_loc

    # Analysis using all data: ["full"]
    plocfull = get_local_probs(pe, tr=False)
    data[i]["full"] = {}
    for ptype in pathtypes:
        data[i]["full"][ptype] = plocfull[ptype]

    if VERBOSE:
        print(f"Finished ensemble {i}")

### Generate pathlength distribution figures, as in PyRETIS reports.

In [ ]:
for i, pe in enumerate(pathensembles):
    upe = pe.unify_pe()
    # Pathlength distribution
    data[i]["pathlengths"] = pathlength_distr(upe)  # these might be used later or not! TODO
        
#=======================================
# make figures
makefigs = True 
if makefigs:
    for i, pe in enumerate(pathensembles):     
        if i == 0:
            continue
        # Cross distances distribution
        L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(pe)
        fig,ax = plt.subplots()
        ax.plot(lmllambs, lmlpercs, lw=1, c="g")
        ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
        for lamb in (L,M,R):
            ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
        ax.set_xlabel('Cross distance')
        ax.set_ylabel('Frequency')
        ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
            pe.name, L, M, R))
        ax.set_ylim(0)
        fig.savefig(f"pathensemble_{i}_crossdist.pdf")
        plt.close(fig)

        # Pathlength distribution      
        for ptype in pathtypes:
            fig, ax = plt.subplots()
            ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
                data[i]["pathlengths"][ptype]["hist"])
            ax.set_xlabel('Pathlength')
            ax.set_ylabel('Frequency')
            ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
                         f"{ptype} paths. ")
            ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
                          f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
            fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
            plt.close(fig)

### Compute Pcross using in-house functions

In [ ]:
# Global crossing probabilities (no error analysis)  
psfull = []
for i in range(1, len(pathensembles)):   # do not use the 0- ensemble
    psfull.append({"LMR": data[i]["full"]["LMR"], 
               "RML": data[i]["full"]["RML"], 
               "RMR": data[i]["full"]["RMR"],
               "LML": data[i]["full"]["LML"]})

Pminfull, Pplusfull, Pcrossfull = get_global_probs_from_dict(psfull)

In [ ]:
# Make a figure of the global crossing probabilities
fig, ax = plt.subplots()
ax.set_yscale("log")
# ax.plot(Pcrossfull, "o", c = "r")
ax.errorbar(interfaces, Pcrossfull, fmt="-o", c = "b", ecolor="r", capsize=6)


ax.set_xlabel("OP")
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
fig.tight_layout()
fig.show()
fig.savefig("Global_probs.pdf")

print("This should be the same as the repptis_report.pdf value:", Pcrossfull[-1])
print("which is the case!")
print(Pcrossfull)
print([Pcrossfull[i]/Pcrossfull[i-1] for i in range(1,len(Pcrossfull))])
print("Here, the load immediately disappeared. For a simulation where this is")
print("not the case, the above code should be adapted a little bit.")

In [ ]:
# Construct lists of the local probs

# Or we can use the get_global_probs_from_local function, using lists of the local probs
# These do not use the 0- ensemble
pmps = [data[i]["full"]["LMR"] for i in range(1,len(pathensembles))]
pmms = [data[i]["full"]["LML"] for i in range(1,len(pathensembles))]
ppps = [data[i]["full"]["RMR"] for i in range(1,len(pathensembles))]
ppms = [data[i]["full"]["RML"] for i in range(1,len(pathensembles))]
a,b,c = get_global_probs_from_local(pmps, pmms, ppps, ppms)
print("This should be the same as the repptis_report.pdf value:", c[-1])

## 4. Analysis using the MSM

### Construct transition matrix M

In [ ]:
print(interfaces)
N = len(interfaces)
NS = 4*N-5
print("N", N)
# print("len pmms", len(pmms)) # TODO INCLUDE?
print("NS", NS)

labels1, labels2 = create_labels_states(N)

In [ ]:
if VERBOSE:
    print("mm", pmms)
    print("mp", pmps)
    print("pm", ppms)
    print("pp", ppps)
    print("sum", np.array(pmms) + np.array(pmps))
    print("sum", np.array(ppms) + np.array(ppps))

M = construct_M(pmms, pmps, ppms, ppps, N)

In [ ]:
# We can print the transition matrix M and check that all rows sum to 1.
if VERBOSE:
    print("M")
    print("shape", M.shape)
    print("sum prob in rows", np.sum(M, axis=1))
    print(M)
else:
    print("Transition matrix built. Set VERBOSE=True to print matrix and row sums.")

### Look at this Markov model

In [ ]:
if VERBOSE:
    vals, vecs = np.linalg.eig(M)
    print(vals)
    vals, vecs = np.linalg.eig(M.T)
    print(vals)
    pprint(M)
else:
    vals, _ = np.linalg.eig(M)
    print(f"Eigenvalues (summary): min={np.min(vals):.3f}, max={np.max(vals):.3f}")

In [ ]:
if VERBOSE:
    print("what if chain propagates")
    print("A[0,:]")
    # check stationary behavior
    A = M
    for n in range(10):
        A = np.dot(A, M)
        print(A[0, :])
        print(np.sum(A[0, :]))  # is 1 indeed
else:
    print("Skipping propagation printout (set VERBOSE=True to inspect A^n)")

### Pcross with MSM

In [ ]:
# Inspect Z and Y vectors

z1, z2, y1, y2 = global_pcross_msm(M)
print("Z")
print_vector(z1, labels1)
print_vector(z2, labels2)
print("Y")
print_vector(y1, labels1)
print_vector(y2, labels2)
print("\nGlobal crossing probability: ", y1[0][0])

### Pathlength analysis

In [ ]:
# Setting path ensemble properties
#==================================
for i,fol in enumerate(folders):
    print(i)
    print("Calculating path lengths.")
    set_tau_distrib(pathensembles[i])
    print("Done.")

    if True:
        print("Calculating first hitting lengths to middle interface")
        set_tau_first_hit_M_distrib(pathensembles[i])
        print("Done.")

In [ ]:
# Additional information
#==================================
# Average path lengths per ensemble for each path type
print(indir[-20:])
pathtypes = ("LML", "LMR", "RMR", "RML", "LM*", "*M*", "***", "RM*", "L**", "**R", "R**")

print("=" * 80)
print("AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE")
print("=" * 80)

for i, pe in enumerate(pathensembles):
    if not VERBOSE:
        print(f"Ensemble {i}: accepted paths = {np.sum(np.isin(pe.flags, ACCFLAGS))}")
        continue

    print(f"\nEnsemble {i} ({pe.name}):")
    print("-" * 50)
    
    # Get accepted paths only
    accepted_mask = np.isin(pe.flags, ACCFLAGS)
    total_accepted_count = np.sum(accepted_mask)
    
    if total_accepted_count > 0:
        # Calculate weighted average for all accepted paths
        accepted_lengths = pe.lengths[accepted_mask]
        accepted_weights = pe.weights[accepted_mask]
        total_weighted_avg = np.average(accepted_lengths, weights=accepted_weights)
        
        print(f"  All accepted paths: {total_weighted_avg:8.2f} (n={total_accepted_count:4d}, weighted)")
        print("-" * 30)
    
    for ptype in pathtypes:
        mask = (pe.lmrs == ptype) & accepted_mask
        if np.any(mask):
            lengths = pe.lengths[mask]
            weights = pe.weights[mask]
            weighted_avg = np.average(lengths, weights=weights)
            count = np.sum(mask)
            print(f"  {ptype:4s}: {weighted_avg:8.2f} (n={count:4d}, weighted)")
        else:
            print(f"  {ptype:4s}: {0:8.2f} (n={0:4d}, weighted)")

In [ ]:
# Compute taus for pathlength analysis
tau_mm, tau_mp, tau_pm, tau_pp = collect_tau(pathensembles)
tau1_mm, tau1_mp, tau1_pm, tau1_pp = collect_tau1(pathensembles)
tau2_mm, tau2_mp, tau2_pm, tau2_pp = collect_tau2(pathensembles)
taum_mm, taum_mp, taum_pm, taum_pp = collect_taum(pathensembles)

In [ ]:
# Look at computed taus
if VERBOSE:
    print("tau")
    print_all_tau(pathensembles, tau_mm, tau_mp, tau_pm, tau_pp)
    print("\ntau1")
    print_all_tau(pathensembles, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
    print("\ntaum")
    print_all_tau(pathensembles, taum_mm, taum_mp, taum_pm, taum_pp)
    print("\ntau2")
    print_all_tau(pathensembles, tau2_mm, tau2_mp, tau2_pm, tau2_pp)
else:
    print("Tau diagnostics suppressed (set VERBOSE=True for details)")

In [ ]:
# TODO include prints?
tau  = construct_tau_vector(N, NS, tau_mm, tau_mp, tau_pm, tau_pp)
tau1 = construct_tau_vector(N, NS, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
taum = construct_tau_vector(N, NS, taum_mm, taum_mp, taum_pm, taum_pp)
tau2 = construct_tau_vector(N, NS, tau2_mm, tau2_mp, tau2_pp)
tau_m = tau - tau1 - tau2  # yes, this is the same thing as taum

if VERBOSE:
    print("tau")
    print(tau)
    print("\n")
    print("tau1")
    print(tau1)
    print("taum")
    print(taum)
    print("tau2")
    print(tau2)
    print("\n")
    print("tau = tau1+taum+tau2 => difference is", np.sum((tau - tau1 - taum - tau2) ** 2))
else:
    diff = np.sum((tau - tau1 - taum - tau2) ** 2)
    print(f"Tau vectors built. Consistency diff: {diff:.3e} (set VERBOSE=True to inspect)")

## 5. Flux calculation

### Collect tau for [0+]

In [ ]:
# Construct g and h vectors
g1, g2, h1, h2 = mfpt_to_first_last_state(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2)) #, doprint=True)
print("G")
print_vector(g1, labels1)
print_vector(g2, labels2)
print("H")
print_vector(h1, labels1)
print_vector(h2, labels2)
print("\ntau [0+]: ", h1[0])

### The flux

In [ ]:
flux = 1/(tau[0]+h1[0][0])
print(tau[0], h1[0][0])
dt = 0.002 # Change if needed
sc = 10
flux /= (dt*sc)
print(flux*(dt*sc), "1/time")
print(flux, "1/ps")

## 6. The rate constant
We can compute an accurate rate constant using only our MSM.

In [ ]:
# rate constant = flux * Pcross

print("The rate constant k is: ", flux*y1[0][0]/dt)

## 7. Direct rate computation via $\tau_{\mathcal{A},1}$ 

In [ ]:
# Construct g and h vectors
absor = np.array([NS - 1])
kept = np.array([i for i in range(NS) if i not in absor])

g1, g2, h1, h2 = mfpt_to_absorbing_states(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2), absor, kept, remove_initial_m=False) #, doprint=True)
print("G")
print_vector(g1, labels1[-1])
print_vector(g2, [labels1[0]] + labels2)
print("H")
print_vector(h1, labels1[-1])
print_vector(h2, [labels1[0]] + labels2)
print("interesting")
print(h2[0])
mfpt = h2[0][0]  # tau_A,1

In [ ]:
k_flux_pcross = flux * y1[0][0] / dt
k_mfpt = 1 / (mfpt * dt)

print(f"Rate constant from P_cross × flux: {k_flux_pcross:.10e} [1/ps]")
print(f"Rate constant from MFPT:           {k_mfpt:.10e} [1/ps]")
print(f"Relative difference:               {abs(k_flux_pcross - k_mfpt)/k_mfpt*100:.2f}%")
